# Setup

In [13]:
from google.colab import drive
import pandas as pd

# Technical Indicators


In [14]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:

file_path = "/content/drive/MyDrive/24fall_dlProject/data/BTC-USD_stock_data.csv"

data = pd.read_csv(file_path,index_col='Date',parse_dates=['Date'])

In [16]:
data[:10]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-01,963.658020,1003.080017,958.698975,998.325012,998.325012,147775008
2017-01-02,998.617004,1031.390015,996.702026,1021.750000,1021.750000,222184992
2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966,1043.839966,185168000
2017-01-04,1044.400024,1159.420044,1044.400024,1154.729980,1154.729980,344945984
2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,1013.380005,510199008
2017-01-06,1014.239990,1046.810059,883.943970,902.200989,902.200989,351876000
2017-01-07,903.487000,908.585022,823.556030,908.585022,908.585022,279550016
2017-01-08,908.174988,942.723999,887.249023,911.198975,911.198975,158715008
2017-01-09,913.244019,913.685974,879.807007,902.828003,902.828003,141876992


In [17]:
def sma(data, window):
  return data.rolling(window=window).mean()

def rsi(prices, period):

  delta = prices.diff()

  gain = delta.where(delta > 0, 0)
  loss = -delta.where(delta < 0, 0)

  avg_gain = gain.rolling(window=period).mean()
  avg_loss = loss.rolling(window=period).mean()

  rs = avg_gain / avg_loss

  rsi = 100 - (100 / (1 + rs))
  return rsi

def kdj(data, period=9, alpha=1/3):

  data['Ln'] = data['Low'].rolling(window=period).min()
  data['Hn'] = data['High'].rolling(window=period).max()
  data['RSV'] = (data['Close'] - data['Ln']) / (data['Hn'] - data['Ln']) * 100

  data['K'] = data['RSV'].ewm(alpha=alpha, adjust=False).mean()
  data['D'] = data['K'].ewm(alpha=alpha, adjust=False).mean()

  data['J'] = 3 * data['K'] - 2 * data['D']

  data = data.drop(columns=['Ln', 'Hn', 'RSV'])

  return data

def obv(data):

  obv = [0]
  for i in range(1, len(data)):
      if data['Close'].iloc[i] > data['Close'].iloc[i - 1]:
          obv.append(obv[-1] + data['Volume'].iloc[i])
      elif data['Close'].iloc[i] < data['Close'].iloc[i - 1]:
           obv.append(obv[-1] - data['Volume'].iloc[i])
      else:
          obv.append(obv[-1])
  return pd.Series(obv, index=data.index)

def bias(data, period):

  n_day_avg = data['Close'].rolling(window=period).mean()
  bias = ((data['Close'] - n_day_avg) / n_day_avg) * 100
  return bias

data['SMA (fast)'] = sma(data['Close'], 5)
data['SMA (middle)'] = sma(data['Close'], 10)
data['SMA (slow)'] = sma(data['Close'], 20)
data['RSI (slow)'] = rsi(data['Close'], 24)
data['RSI (middle)'] = rsi(data['Close'], 12)
data['RSI (fast)'] = rsi(data['Close'], 6)
data = kdj(data, period=9)
data['OBV'] = obv(data)
data['BIAS (slow)'] = bias(data, period=12)
data['BIAS (middle)'] = bias(data, period=10)
data['BIAS (fast)'] = bias(data, period=5)

In [26]:
output_file_path = '/content/drive/MyDrive/24fall_dlProject/data/BTC-USD_stock_data_with_indicators.csv'
data.to_csv(output_file_path)

In [19]:
data[:10]

,Open,High,Low,Close,Adj Close,Volume,SMA (fast),SMA (middle),SMA (slow),RSI (slow),RSI (middle),RSI (fast),K,D,J,OBV,BIAS (slow),BIAS (middle),BIAS (fast)
Date,,,,,,,,,,,,,,,,,,,
2017-01-01,963.658020,1003.080017,958.698975,998.325012,998.325012,147775008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
2017-01-02,998.617004,1031.390015,996.702026,1021.750000,1021.750000,222184992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222184992,NaN,NaN,NaN
2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966,1043.839966,185168000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,407352992,NaN,NaN,NaN
2017-01-04,1044.400024,1159.420044,1044.400024,1154.729980,1154.729980,344945984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,752298976,NaN,NaN,NaN
2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,1013.380005,510199008,1046.404993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242099968,NaN,NaN,-3.156043
2017-01-06,1014.239990,1046.810059,883.943970,902.200989,902.200989,351876000,1027.180188,NaN,NaN,NaN,NaN,38.246999,NaN,NaN,NaN,-109776032,NaN,NaN,-12.167213
2017-01-07,903.487000,908.585022,823.556030,908.585022,908.585022,279550016,1004.547192,NaN,NaN,NaN,NaN,39.196231,NaN,NaN,NaN,169773984,NaN,NaN,-9.552779
2017-01-08,908.174988,942.723999,887.249023,911.198975,911.198975,158715008,978.018994,NaN,NaN,NaN,NaN,35.988710,NaN,NaN,NaN,328488992,NaN,NaN,-6.832180
2017-01-09,913.244019,913.685974,879.807007,902.828003,902.828003,141876992,927.638599,NaN,NaN,NaN,NaN,31.484189,21.568026,21.568026,21.568026,186612000,NaN,NaN,-2.674597


In [20]:
file_path1 = "/content/drive/MyDrive/24fall_dlProject/data/spy.csv"

data1 = pd.read_csv(file_path1,index_col='Date',parse_dates=['Date'])

In [21]:
data1[:10]

,Open,High,Low,Close,Volume,Day,Weekday,Week,Month,Year
Date,,,,,,,,,,
1993-01-29,24.626128,24.626128,24.503610,24.608625,1003200,29,4,4,1,1993
1993-02-01,24.626124,24.783648,24.626124,24.783648,480500,1,0,5,2,1993
1993-02-02,24.766156,24.853669,24.713648,24.836166,201300,2,1,5,2,1993
1993-02-03,24.871164,25.116200,24.853662,25.098698,529400,3,2,5,2,1993
1993-02-04,25.186225,25.256235,24.906184,25.203728,531500,4,3,5,2,1993
1993-02-05,25.186205,25.238713,25.046184,25.186205,492100,5,4,5,2,1993
1993-02-08,25.186205,25.273718,25.151200,25.186205,596100,8,0,6,2,1993
1993-02-09,25.098699,25.098699,24.958679,25.011187,122100,9,1,6,2,1993
1993-02-10,25.011179,25.063687,24.941169,25.046185,379600,10,2,6,2,1993


In [22]:
data1['SMA (fast)'] = sma(data1['Close'], 5)
data1['SMA (middle)'] = sma(data1['Close'], 10)
data1['SMA (slow)'] = sma(data1['Close'], 20)
data1['RSI (slow)'] = rsi(data1['Close'],24)
data1['RSI (middle)'] = rsi(data1['Close'],12)
data1['RSI (fast)'] = rsi(data1['Close'],6)
data1 = kdj(data1, period=9)
data1['OBV'] = obv(data1)
data1['BIAS (slow)'] = bias(data1, period=12)
data1['BIAS (middle)'] = bias(data1, period=10)
data1['BIAS (fast)'] = bias(data1, period=5)

In [23]:
data1 = data1.drop(columns=['Day'])
data1 = data1.drop(columns=['Weekday'])
data1 = data1.drop(columns=['Week'])
data1 = data1.drop(columns=['Month'])
data1 = data1.drop(columns=['Year'])

In [24]:
data1[:10]

,Open,High,Low,Close,Volume,SMA (fast),SMA (middle),SMA (slow),RSI (slow),RSI (middle),RSI (fast),K,D,J,OBV,BIAS (slow),BIAS (middle),BIAS (fast)
Date,,,,,,,,,,,,,,,,,,
1993-01-29,24.626128,24.626128,24.503610,24.608625,1003200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
1993-02-01,24.626124,24.783648,24.626124,24.783648,480500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,480500,NaN,NaN,NaN
1993-02-02,24.766156,24.853669,24.713648,24.836166,201300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,681800,NaN,NaN,NaN
1993-02-03,24.871164,25.116200,24.853662,25.098698,529400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1211200,NaN,NaN,NaN
1993-02-04,25.186225,25.256235,24.906184,25.203728,531500,24.906173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1742700,NaN,NaN,1.194703
1993-02-05,25.186205,25.238713,25.046184,25.186205,492100,25.021689,NaN,NaN,NaN,NaN,97.139717,NaN,NaN,NaN,1250600,NaN,NaN,0.657494
1993-02-08,25.186205,25.273718,25.151200,25.186205,596100,25.102200,NaN,NaN,NaN,NaN,97.139717,NaN,NaN,NaN,1250600,NaN,NaN,0.334650
1993-02-09,25.098699,25.098699,24.958679,25.011187,122100,25.137204,NaN,NaN,NaN,NaN,68.570939,NaN,NaN,NaN,1128500,NaN,NaN,-0.501320
1993-02-10,25.011179,25.063687,24.941169,25.046185,379600,25.126702,NaN,NaN,NaN,NaN,67.645607,70.454366,70.454366,70.454366,1508100,NaN,NaN,-0.320445


In [25]:
output_file_path1 = '/content/drive/MyDrive/24fall_dlProject/data/spy_with_indicators.csv'
data1.to_csv(output_file_path1)